# Problem Description

We want to find out how the solution of our inverse problem converges as we increase the number of points for both the new and traditional methods of data interpolation.

If we have what is known as **"posterior consistency"** then we expect that the error in our solution, when compared to the true solution, will always decrease as we increase the number of points we are assimilating.

## Posterior Consistency **NEEDS WORK**

From a Bayesian point of view, the regularisation we choose and the weighting we give it encode information about our assumed prior probability distribution of $q$ before we start assimilating data (adding observations).
Take, for example, the regularisation used in the this problem

$$
\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx
$$

which asserts a prior that the solution $q$ which minimises $J$ should be smooth and gives a weighting $\alpha$ to the assertion.
If we have posterior consistency, the contribution of increasing numbers of measurements $u_{obs}$ should increase the weighting of our data relative to our prior and we should converge towards the true solution.

## Hypothesis

Our two methods minimise two different functionals. 
The first minimises $J$

$$J[u, q] = 
\underbrace{\frac{1}{2}\int_{\Omega_v}\left(\frac{u_{obs} - I(u, \text{P0DG}(\Omega_v))}{\sigma}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx}_{\text{regularization}}$$

whilst the second minimises $J'$

$$J'[u, q] = 
\underbrace{\frac{1}{2}\int_{\Omega}\left(\frac{u_{interpolated} - u}{\sigma}\right)^2dx}_{\text{model-data misfit}} + 
\underbrace{\frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx}_{\text{regularization}}.$$

As set up here increasing the number of points to assimilate has the effect of increasing the size of the misfit term in $J$ (with a weight proportional to each measurement's variance $\sigma$ so we expect to converge to $q_{true}$ as the number of measurements increases.

As we increase the number of measurements in $J'$ we have to hope that (a) our calculated $u_{interpolated}$ approaches $u$ (to minimise the misfit) and (b) we do not expect the misfit term to increase relative to the regularizatuion term since it doesn't get relatively bigger.

I therefore predict that minimising $J$ will display posterior consistency and that minimising the various $J'$ for each $u_{interpolated}$ will not.
Who knows what we will converge to!

## Hypothesis Amenendment! A note on finite element method error
Note that our solutions all exist in finite element spaces which are usually approximations of a true solution with some error that (hopefully) decreases as mesh density increase and solution space order increase.
Since I am comparing to a solution $u_true$ in CG2 space I expect, at best, that we will converge to $u_true$ when we have, on average, enough points per cell to fully specify the lagrange polynomials in that cell.
Were we in CG1 this would be 3 points per cell (I can't remember how many we would need for CG2!) to give convergence if those measurements had no noise.
Since our measurements are noisy I do not expect actual convergence, but I anticipate some slowing in convergence.

## Change to the misfit functional

To speed things up (see `poisson-inverse-conductivity-investigation.ipynb`) I multiply the misfit through by $\sigma^2$.

# Setup

In [ ]:
import firedrake
import firedrake_adjoint
mesh = firedrake.UnitSquareMesh(32, 32)
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

## Fake $q_{true}$

In [ ]:
from firedrake import Constant, cos, sin
import numpy as np
from numpy import pi as π
from numpy import random
import matplotlib.pyplot as plt

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])

        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z
        
        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)
        
        q_true += mode
        
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(q_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

## Fake $u_{true}$

In [ ]:
from firedrake import exp, inner, grad, dx
u_true = firedrake.Function(V)
v = firedrake.TestFunction(V)
f = Constant(1.0)
k0 = Constant(0.5)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q_true) * inner(grad(u_true), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u_true, bc)

fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(u_true, axes=axes, shading='gouraud')
fig.colorbar(colors);

## Generating Observational Data $u_{obs}$
We run up in powers of 2 until we have plenty of observations per cell (on average)

In [ ]:
min_power_of_2 = 2
max_power_of_2 = 2
signal_to_noise = 20
U = u_true.dat.data_ro[:]
u_range = U.max() - U.min()
σ = firedrake.Constant(u_range / signal_to_noise)

xs_set = {}
u_obs_vals_set = {}

for i in range(min_power_of_2, max_power_of_2+1):
    
    # Make random point cloud
    num_points = 2**i
    xs = np.random.random_sample((num_points,2))
    xs_set[i] = xs
    
    # Generate "observed" data
    ζ = generator.standard_normal(len(xs))
    u_obs_vals = np.array(u_true.at(xs)) + float(σ) * ζ
    u_obs_vals_set[i] = u_obs_vals
    
print(2**max_power_of_2 / mesh.num_cells())

# Solving with Vertex Only Meshes

In [ ]:
q_min_set = {}

for i in range(min_power_of_2, max_power_of_2+1):
    
    # Run the forward problem with q = 0 as first guess
    u = firedrake.Function(V)
    q = firedrake.Function(Q)
    bc = firedrake.DirichletBC(V, 0, 'on_boundary')
    F = (k0 * exp(q) * inner(grad(u), grad(v)) - f * v) * dx
    firedrake.solve(F == 0, u, bc)
    
    # Store data on the point_cloud using a vertex only mesh
    point_cloud = firedrake.VertexOnlyMesh(mesh, xs_set[i])
    P0DG = firedrake.FunctionSpace(point_cloud, 'DG', 0)
    u_obs = firedrake.Function(P0DG)
    u_obs.dat.data[:] = u_obs_vals_set[i]

    # Two terms in the functional
    misfit_expr = 0.5 * ((u_obs - firedrake.interpolate(u, P0DG)) / σ)**2
    α = firedrake.Constant(0.5)
    regularisation_expr = 0.5 * α**2 * inner(grad(q), grad(q))

    # Should be able to write firedrake.assemble(misfit + regularisation * dx) but can't yet 
    # because of the meshes being different
    J = firedrake.assemble(misfit_expr * dx) + firedrake.assemble(regularisation_expr * dx)
    
    # Create reduced functional
    q̂ = firedrake_adjoint.Control(q)
    Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)
    
    # Minimise reduced functional
    q_min = firedrake_adjoint.minimize(
        Ĵ, method='Newton-CG', options={'disp': True}
    )
    q_min_point_cloud = {}
    q_min_point_cloud['point_cloud'] = q_min
    q_min_set[i] = q_min_point_cloud.copy()

In [ ]:
# xs = xs_set[2]
# q_min = point_cloud_q_min[2]
# fig, axes = plt.subplots(ncols=3, nrows=2, sharex=True, sharey=True, figsize=(20,12), dpi=200)
# plt.suptitle('Estimating Log-Conductivity $q$ \n\
# where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$', fontsize=25)
# for ax in axes.ravel():
#     ax.set_aspect('equal')
# #     ax.get_xaxis().set_visible(False)

# axes[0, 0].set_title('$u_{true}$', fontsize=25)
# colors = firedrake.tripcolor(u_true, axes=axes[0, 0], shading='gouraud')
# fig.colorbar(colors, ax=axes[0, 0])
# axes[1, 0].set_title('Sampled Noisy $u_{obs}$', fontsize=25)
# colors = axes[1, 0].scatter(xs[:, 0], xs[:, 1], c=u_obs_vals)
# fig.colorbar(colors, ax=axes[1, 0])

# kw = {'vmin': -5, 'vmax': +5, 'shading': 'gouraud'}
# axes[0, 1].set_title('$q_{true}$', fontsize=25)
# colors = firedrake.tripcolor(q_true, axes=axes[0, 1], **kw)
# fig.colorbar(colors, ax=axes[0, 1])
# axes[1, 1].set_title('Estimated $q_{est}$ from $u_{obs}$', fontsize=25)
# colors = firedrake.tripcolor(q_min, axes=axes[1, 1], **kw);
# fig.colorbar(colors, ax=axes[1, 1])

# axes[0, 2].axis('off')
# q_err = firedrake.Function(Q).assign(q_min-q_true)
# l2norm = firedrake.norm(q_err, "L2")
# axes[1, 2].set_title('$q_{est}$ - $q_{true}$', fontsize=25)
# axes[1, 2].text(0.5, 0.5, f'$L^2$ Norm {l2norm:.2f}', ha='center', fontsize=20)
# colors = firedrake.tripcolor(q_err, axes=axes[1, 2], **kw);
# fig.colorbar(colors, ax=axes[1, 2])

# Solving with Interpolation Methods

In [ ]:
from scipy.interpolate import (
    LinearNDInterpolator,
    NearestNDInterpolator,
    CloughTocher2DInterpolator,
    Rbf,
)

interpolators_set = {}

for i in range(min_power_of_2, max_power_of_2+1):
    interpolators_set[i] = {
        'nearest': NearestNDInterpolator(xs, u_obs_vals),
        'linear': LinearNDInterpolator(xs, u_obs_vals, fill_value=0.0),
        'clough-tocher': CloughTocher2DInterpolator(xs, u_obs_vals, fill_value=0.0),
        'gaussian': Rbf(xs[:, 0], xs[:, 1], u_obs_vals, function='gaussian'),
    }

In [ ]:
interpolated_data_set = {}

for i in range(min_power_of_2, max_power_of_2+1):
    # Interpolating the mesh coordinates field (which is a vector function space) 
    # into the vector function space equivalent of our solution space gets us 
    # global DOF values (stored in the dat) which are the coordinates of the global
    # DOFs of our solution space. This is the necessary coordinates field X.
    Vc = firedrake.VectorFunctionSpace(mesh, V.ufl_element())
    X = firedrake.interpolate(mesh.coordinates, Vc).dat.data_ro[:]

    # Interpolate using each method
    interpolated_data = {}
    for method, interpolator in interpolators_set[i].items():
        u_interpolated = firedrake.Function(V)
        u_interpolated.dat.data[:] = interpolator(X[:, 0], X[:, 1])
        interpolated_data[method] = u_interpolated
        
    # Save interpolated data for number of points
    interpolated_data_set[i] = interpolated_data

In [ ]:
for i in range(min_power_of_2, max_power_of_2+1):

    for method, u_interpolated in interpolated_data.items():

        # Run the forward problem with q = 0 as first guess
        u = firedrake.Function(V)
        q = firedrake.Function(Q)
        bc = firedrake.DirichletBC(V, 0, 'on_boundary')
        F = (k0 * exp(q) * inner(grad(u), grad(v)) - f * v) * dx
        firedrake.solve(F == 0, u, bc)

        # Two terms in the functional
        misfit_expr = 0.5 * ((u_interpolated - u) / σ)**2
        α = firedrake.Constant(0.5)
        regularisation_expr = 0.5 * α**2 * inner(grad(q), grad(q))

        # Only assemble two terms separately for exact comparison with other method!
        Jprime = firedrake.assemble(misfit_expr * dx) + firedrake.assemble(regularisation_expr * dx)

        # Create reduced functional
        q̂ = firedrake_adjoint.Control(q)
        Ĵprime = firedrake_adjoint.ReducedFunctional(Jprime, q̂)

        # Minimise reduced functional
        q_min = firedrake_adjoint.minimize(
            Ĵprime, method='Newton-CG', options={'disp': True}
        )

        q_min_set[i][method] = q_min

# Results

# Collate Results

In [ ]:
q_err_set = {}
l2errors_set = {}
for i in range(min_power_of_2, max_power_of_2+1):
    q_err_set[i] = {}
    l2errors_set[i] = {}
    for method_i, q_min_i in q_min_set[i].items():
        q_err = firedrake.Function(Q).assign(q_min_i-q_true)
        l2norm = firedrake.norm(q_err, "L2")
        q_err_set[i][method_i] = q_err
        l2errors_set[i][method_i] = l2norm
        print(method_i)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

for i in range(min_power_of_2, max_power_of_2+1):

    ukw = {'vmin': 0.0, 'vmax': +0.2}
    kw = {'vmin': -4, 'vmax': +4, 'shading': 'gouraud'}
    title_fontsize = 20
    text_fontsize = 20


    fig, axes = plt.subplots(ncols=3, nrows=6, sharex=True, sharey=True, figsize=(20,30), dpi=200)
    plt.suptitle('Estimating Log-Conductivity $q$ \n\
    where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$', fontsize=title_fontsize)
    for ax in axes.ravel():
        ax.set_aspect('equal')
    #     ax.get_xaxis().set_visible(False)

    axes[0, 0].set_title('$u_{true}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(u_true, axes=axes[0, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[0, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[0, 1].set_title('$q_{true}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_true, axes=axes[0, 1], **kw)
    cax = make_axes_locatable(axes[0, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[0, 2].set_title('$q_{true}-q_{true}$', fontsize=title_fontsize)
    zero_func = firedrake.Function(Q).assign(q_true-q_true)
    axes[0, 2].text(0.5, 0.5, f'$L^2$ Norm {firedrake.norm(zero_func, "L2"):.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(zero_func, axes=axes[1, 2], **kw);
    cax = make_axes_locatable(axes[0, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'point-cloud'

    axes[1, 0].set_title('Sampled Noisy $u_{obs}$', fontsize=title_fontsize)
    colors = axes[1, 0].scatter(xs[:, 0], xs[:, 1], c=u_obs_vals, vmin=0.0, vmax=0.2)
    cax = make_axes_locatable(axes[1, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[1, 1].set_title('$q_{est}$ from Point Cloud', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[1, 1], **kw)
    cax = make_axes_locatable(axes[1, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[1, 2].set_title('$q_{est}-q_{true}$', fontsize=title_fontsize)
    axes[1, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[1, 2], **kw);
    cax = make_axes_locatable(axes[1, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'nearest'

    axes[2, 0].set_title('$u_{interpolated}^{nearest}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[2, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[2, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[2, 1].set_title('$q_{est}^{nearest}$ from $u_{interpolated}^{nearest}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[2, 1], **kw)
    cax = make_axes_locatable(axes[2, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[2, 2].set_title('$q_{est}^{nearest}-q_{true}$', fontsize=title_fontsize)
    axes[2, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[2, 2], **kw);
    cax = make_axes_locatable(axes[2, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'linear'

    axes[3, 0].set_title('$u_{interpolated}^{linear}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[3, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[3, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[3, 1].set_title('$q_{est}^{linear}$ from $u_{interpolated}^{linear}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[3, 1], **kw)
    cax = make_axes_locatable(axes[3, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[3, 2].set_title('$q_{est}^{linear}-q_{true}$', fontsize=title_fontsize)
    axes[3, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[3, 2], **kw);
    cax = make_axes_locatable(axes[3, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'clough-tocher'

    axes[4, 0].set_title('$u_{interpolated}^{clough-tocher}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[4, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[4, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[4, 1].set_title('$q_{est}^{clough-tocher}$ from $u_{interpolated}^{clough-tocher}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[4, 1], **kw)
    cax = make_axes_locatable(axes[4, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[4, 2].set_title('$q_{est}^{clough-tocher}-q_{true}$', fontsize=title_fontsize)
    axes[4, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[4, 2], **kw);
    cax = make_axes_locatable(axes[4, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'gaussian'

    axes[5, 0].set_title('$u_{interpolated}^{gaussian}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[5, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[5, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[5, 1].set_title('$q_{est}^{gaussian}$ from $u_{interpolated}^{gaussian}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[5, 1], **kw)
    cax = make_axes_locatable(axes[5, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[5, 2].set_title('$q_{est}^{gaussian}-q_{true}$', fontsize=title_fontsize)
    axes[5, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[5, 2], **kw);
    cax = make_axes_locatable(axes[5, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    # fig.text(0.5,0.05,r'Functional minimised: $J[u, q] = \frac{1}{2}\int_{\Omega_v}\left(\frac{u_{obs} - I(u, \mathrm{P0DG}(\Omega_v))}{\sigma}\right)^2dx + \frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx$', ha='center', va='center', fontsize=20)

    plt.savefig(f'posterior-consistency-{2**i}-pts.png')

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

for i in range(min_power_of_2, max_power_of_2+1):

    ukw = {}
    kw = {'shading': 'gouraud'}
    title_fontsize = 20
    text_fontsize = 20


    fig, axes = plt.subplots(ncols=3, nrows=6, sharex=True, sharey=True, figsize=(20,30), dpi=200)
    plt.suptitle('Estimating Log-Conductivity $q$ \n\
    where $k = k_0e^q$ and $-\\nabla \\cdot k \\nabla u = f$ for known $f$', fontsize=title_fontsize)
    for ax in axes.ravel():
        ax.set_aspect('equal')
    #     ax.get_xaxis().set_visible(False)

    axes[0, 0].set_title('$u_{true}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(u_true, axes=axes[0, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[0, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[0, 1].set_title('$q_{true}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_true, axes=axes[0, 1], **kw)
    cax = make_axes_locatable(axes[0, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[0, 2].set_title('$q_{true}-q_{true}$', fontsize=title_fontsize)
    zero_func = firedrake.Function(Q).assign(q_true-q_true)
    axes[0, 2].text(0.5, 0.5, f'$L^2$ Norm {firedrake.norm(zero_func, "L2"):.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(zero_func, axes=axes[1, 2], **kw);
    cax = make_axes_locatable(axes[0, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'point-cloud'

    axes[1, 0].set_title('Sampled Noisy $u_{obs}$', fontsize=title_fontsize)
    colors = axes[1, 0].scatter(xs[:, 0], xs[:, 1], c=u_obs_vals, vmin=0.0, vmax=0.2)
    cax = make_axes_locatable(axes[1, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[1, 1].set_title('$q_{est}$ from Point Cloud', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[1, 1], **kw)
    cax = make_axes_locatable(axes[1, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[1, 2].set_title('$q_{est}-q_{true}$', fontsize=title_fontsize)
    axes[1, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[1, 2], **kw);
    cax = make_axes_locatable(axes[1, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'nearest'

    axes[2, 0].set_title('$u_{interpolated}^{nearest}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[2, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[2, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[2, 1].set_title('$q_{est}^{nearest}$ from $u_{interpolated}^{nearest}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[2, 1], **kw)
    cax = make_axes_locatable(axes[2, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[2, 2].set_title('$q_{est}^{nearest}-q_{true}$', fontsize=title_fontsize)
    axes[2, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[2, 2], **kw);
    cax = make_axes_locatable(axes[2, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'linear'

    axes[3, 0].set_title('$u_{interpolated}^{linear}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[3, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[3, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[3, 1].set_title('$q_{est}^{linear}$ from $u_{interpolated}^{linear}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[3, 1], **kw)
    cax = make_axes_locatable(axes[3, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[3, 2].set_title('$q_{est}^{linear}-q_{true}$', fontsize=title_fontsize)
    axes[3, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[3, 2], **kw);
    cax = make_axes_locatable(axes[3, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'clough-tocher'

    axes[4, 0].set_title('$u_{interpolated}^{clough-tocher}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[4, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[4, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[4, 1].set_title('$q_{est}^{clough-tocher}$ from $u_{interpolated}^{clough-tocher}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[4, 1], **kw)
    cax = make_axes_locatable(axes[4, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[4, 2].set_title('$q_{est}^{clough-tocher}-q_{true}$', fontsize=title_fontsize)
    axes[4, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[4, 2], **kw);
    cax = make_axes_locatable(axes[4, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)


    key = 'gaussian'

    axes[5, 0].set_title('$u_{interpolated}^{gaussian}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(interpolated_data[key], axes=axes[5, 0], shading='gouraud', **ukw)
    cax = make_axes_locatable(axes[5, 0]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[5, 1].set_title('$q_{est}^{gaussian}$ from $u_{interpolated}^{gaussian}$', fontsize=title_fontsize)
    colors = firedrake.tripcolor(q_min_set[i][key], axes=axes[5, 1], **kw)
    cax = make_axes_locatable(axes[5, 1]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    axes[5, 2].set_title('$q_{est}^{gaussian}-q_{true}$', fontsize=title_fontsize)
    axes[5, 2].text(0.5, 0.5, f'$L^2$ Norm {l2errors_set[i][key]:.2f}', ha='center', fontsize=text_fontsize)
    colors = firedrake.tripcolor(q_err_set[i][key], axes=axes[5, 2], **kw);
    cax = make_axes_locatable(axes[5, 2]).append_axes("right", size="5%", pad=0.05)
    fig.colorbar(colors, cax=cax)

    # fig.text(0.5,0.05,r'Functional minimised: $J[u, q] = \frac{1}{2}\int_{\Omega_v}\left(\frac{u_{obs} - I(u, \mathrm{P0DG}(\Omega_v))}{\sigma}\right)^2dx + \frac{\alpha^2}{2}\int_\Omega|\nabla q|^2dx$', ha='center', va='center', fontsize=20)

    plt.savefig(f'posterior-consistency-{2**i}-pts-freecolors.png')